In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC # 3. Camada Silver - Limpeza e Padronização dos Dados
# MAGIC 
# MAGIC **Objetivo:** Limpar, transformar e enriquecer os dados da camada Bronze.

# COMMAND ----------

from pyspark.sql.functions import col, regexp_replace, to_date, year

# Define o catálogo a ser usado
spark.sql("USE CATALOG movies_catalog")

# COMMAND ----------

# Carrega a tabela da camada Bronze
df_bronze = spark.read.table("bronze.movies_raw")

# Realiza a limpeza e transformação dos dados
df_silver = df_bronze \
    .withColumn("orcamento", regexp_replace(col("budget"), "[$,]", "").cast("long")) \
    .withColumn("receita", regexp_replace(col("revenue"), "[$,]", "").cast("long")) \
    .withColumn("data_lancamento", to_date(col("release_date"), "yyyy-MM-dd")) \
    .withColumn("ano_lancamento", year(col("data_lancamento"))) \
    .withColumnRenamed("original_title", "titulo_original") \
    .withColumnRenamed("vote_average", "nota_media") \
    .withColumnRenamed("runtime", "duracao_min") \
    .drop("budget", "revenue", "release_date", "homepage", "tagline") # Remove colunas antigas ou desnecessárias

# Seleciona e reordena as colunas para a tabela final
df_silver = df_silver.select(
    "id",
    "titulo_original",
    "data_lancamento",
    "ano_lancamento",
    "genres",
    "orcamento",
    "receita",
    "duracao_min",
    "nota_media",
    "overview"
)

silver_table_name = "silver.movies_cleaned"

# Salva a tabela limpa na camada Silver
df_silver.write.mode("overwrite") \
         .option("overwriteSchema", "true") \
         .saveAsTable(silver_table_name)

print(f"Tabela '{silver_table_name}' criada com sucesso na camada Silver.")
display(spark.table(silver_table_name))